In [1]:
#!pip install lightgbm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import joblib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
from sklearn.metrics import mean_absolute_percentage_error as mape_ska
from sklearn.model_selection import train_test_split
from evaluation import predict_multiple,ensemble_output

pd.set_option('display.max_rows', 300)

In [4]:
core = pd.read_csv("core_with_preds.csv", index_col=0)

## Dataset

Adjuntamos al dataset original del Core a febrero 2022, la siguientes variables:
* Datos del bureau de créditos obtenidos en la simulación. 45 variables (ver Anexo)
* net_income_verified: Ingreso declarado en el momento de simulación. Obtenido desde kikoya.
* ingreso_neto_comprobado: Ingreso real obtenido desde netsuite.
* ML_declarado: ingreso predicho con un modelo que tiene en cuenta net_income_verified
* Ml_nodeclardo: ingreso predicho sin esa columna
* ML_income: respuesta de nuestra API = MIN(ML_nodeclarado, ML_income, net_income_verified)


In [5]:
print(f"Nuestro dataset esta compuesto por {core.Consecutivo.nunique()} clientes únicos y {len(core.columns)} variables")

Nuestro dataset esta compuesto por 13793 clientes únicos y 265 variables


In [6]:
cant_declaran_más= len(core[core.net_income_verified > core.ingreso_neto_comprobado])
print(f"La cantidad de clientes que declaran más son {cant_declaran_más} y representan el {(cant_declaran_más/len(core))*100:.2f}% del dataset")

La cantidad de clientes que declaran más son 2861 y representan el 20.52% del dataset


In [7]:
cant_modelo_con_declarado = len(core[core.ML_declarado > core.ingreso_neto_comprobado])
print(f"La cantidad de clientes que el Modelo1 sobre estima son {cant_modelo_con_declarado} y representan el {(cant_modelo_con_declarado/len(core))*100:.2f}% del dataset")

La cantidad de clientes que el Modelo1 sobre estima son 2929 y representan el 21.01% del dataset


In [8]:
cant_modelo_sin = len(core[core.ML_nodeclarado > core.ingreso_neto_comprobado])
print(f"La cantidad de clientes que el Modelo2 sobreestima son {cant_modelo_sin} y representan el {(cant_modelo_sin/len(core))*100:.2f}% del dataset")

La cantidad de clientes que el Modelo2 sobreestima son 7049 y representan el 50.57% del dataset


In [9]:
cant_modelo_final = len(core[core.ML_income > core.ingreso_neto_comprobado])
print(f"La cantidad de clientes que el modelofinal sobreestima son {cant_modelo_final} y representan el {(cant_modelo_final/len(core))*100:.2f}% del dataset")

La cantidad de clientes que el modelofinal sobreestima son 1535 y representan el 11.01% del dataset


## Ingreso declarado

### Construcción de variables para el analisis

In [10]:
select_cap = ['Dependientes','Monto_de_la_mensualidad','BC_Score_',
 'Ingreso_',
 'Ing_Disp']
import re
core["Ingreso_"] = core["Ingreso_"].apply(lambda x: re.sub("[^\d\,]", "", str(x)))
core["Ing_Disp"] = core["Ing_Disp"].apply(lambda x: re.sub("[^\d\,]", "", str(x)))
core["Ingreso_"] = [x.replace(',', '.') for x in core["Ingreso_"]]
core["Ing_Disp"]= [x.replace(',', '.') for x in core["Ing_Disp"]]
core["Ingreso_"] = [x.split(".")[0] for x in core["Ingreso_"]]
core["Ing_Disp"] = [x.split(".")[0] for x in core["Ing_Disp"]]
core["Ing_Disp"] = pd.to_numeric(core["Ing_Disp"])
core["Ingreso_"]  = pd.to_numeric(core["Ingreso_"])
core["Dependientes"] = core["Dependientes"].apply(lambda x: re.sub("[^\w]", "", str(x)))
core["Dependientes"] = core["Dependientes"].replace("nan",0)
core["Dependientes"] = core["Dependientes"].replace("None",0)
core["Dependientes"] = pd.to_numeric(core["Dependientes"])
core["Dependientes"]  = core["Dependientes"].fillna(0)
core["descuentos"] =  core["Ingreso_"] - core["Ing_Disp"]
core["Monto_de_la_mensualidad"] = core["Monto_de_la_mensualidad"].apply(lambda x: re.sub("[^\d\,]", "", str(x)))
core["Monto_de_la_mensualidad"] = [x.replace(',', '.') for x in core["Monto_de_la_mensualidad"]]
core["Monto_de_la_mensualidad"] = pd.to_numeric(core["Monto_de_la_mensualidad"])


In [11]:
core['BC_Score_'] = core['BC_Score_'].apply(lambda x: re.sub("[^\d\,]", "", str(x)))

In [12]:
core['BC_Score_'] = pd.to_numeric(core['BC_Score_'])

## Capacidad de pago Real

<br>

$$
  \text{CAP true} = \frac{\text{Ingreso real} - 2000*Dependientes - Pagos a Bureau}{Mensualidad}
$$

In [13]:
core["capacidad_pago_real"] = (core["ingreso_neto_comprobado"] - core['descuentos']) / (core["Monto_de_la_mensualidad"])
core["CAP_true"]= np.where((core["capacidad_pago_real"]>=2), 1,0)

## Capacidad de Pago declarada

<br>

In [14]:
core["capacidad_pago_declarada"] = (core["net_income_verified"] - core['descuentos'] ) / (core["Monto_de_la_mensualidad"])
core["CAP_declared"]= np.where((core["capacidad_pago_declarada"]>=2), 1,0)


$$
  \text{CAP Declared} = \frac{\text{Ingreso Declarado} - 2000*Dependientes - Pagos a Bureau}{Mensualidad}
$$

## Capacidad de Pago Predicha

<br>


$$
  \text{CAP pred} = \frac{\text{Ingreso Predicho} - 2000*Dependientes - Pagos a Bureau}{Mensualidad}
$$

In [15]:
core["pagos_bureau"] = core["descuentos"] + 2*core["Dependientes"]

In [16]:
core["capacidad_pago_predicha"] = (core["ML_income"] - core['descuentos']) / (core["Monto_de_la_mensualidad"])
core["CAP_pred"]= np.where((core["capacidad_pago_predicha"]>=2), 1,0)

In [17]:
df2 = core.copy()

In [18]:
core[["Consecutivo","Monto_de_la_mensualidad","pagos_bureau","Dependientes","net_income_verified","ingreso_neto_comprobado","ML_income","capacidad_pago_declarada","capacidad_pago_real","capacidad_pago_predicha"]].to_csv("variables_cap.csv")

In [19]:

df2["rule"]= np.where((df2["BC_Score_"]>=680), 1,0)

In [20]:
df2["rule"]= np.where((df2["rule"]==0), "otros","Perfil_X")


In [21]:
#leer excel de adrian y hacer cruce en otro dataset
# agrupar por bins de bureau promedios

In [61]:
adrian = pd.read_csv("2022_03_core_preprocesado.csv")

In [62]:
adrian = adrian[[col for col in adrian.columns if col not in df2.columns]+["Consecutivo"]]

In [111]:
df_udo = pd.merge(df2,adrian, how="inner", left_on="Consecutivo", right_on="Consecutivo")

In [112]:
[col for col in df_udo if "ing" in col]

['ingreso_neto_comprobado',
 'ingreso_hipotecario',
 'ingreso_prestamo',
 'ingreso_tdc',
 'ingreso_tarjetas_no_bancarias']

In [113]:
[col for col in df_udo if "ML" in col]

['ML_declarado', 'ML_nodeclarado', 'ML_income']

In [114]:
core["max_ML"] = core[['ML_declarado', 'ML_nodeclarado']].max(axis=1)

In [115]:
core["capacidad_max"] = (core["max_ML"]  - core['descuentos']) / (core["Monto_de_la_mensualidad"])
core["CAP_max_pred"]= np.where((core["capacidad_pago_predicha"]>=1.8), 1,0)

In [116]:
df_udo["max_ML"] = df_udo[['ML_declarado', 'ML_nodeclarado']].max(axis=1)
df_udo["capacidad_max"] = (df_udo["max_ML"]  - df_udo['descuentos']) / (df_udo["Monto_de_la_mensualidad"])
df_udo["CAP_max_pred"]= np.where((df_udo["capacidad_pago_predicha"]>=1.8), 1,0)

In [117]:
df_udo["CAP_max_pred"]= np.where((df_udo["CAP_max_pred"]==0), "Desaprobado","Aprobado")

# Salvando los perfil x

In [118]:
perfilx = core[core["BC_Score_"]>=680]

In [119]:
(perfilx["CAP_declared"] == 0).sum()

339

In [120]:
len(perfilx[(perfilx["CAP_declared"] == 1) & (perfilx["CAP_true"] == 0)])

116

In [121]:
len(perfilx[(perfilx["CAP_declared"] == 1) & (perfilx["CAP_true"] == 0)& (perfilx["CAP_pred"] == 1)])

96

In [122]:
len(perfilx[(perfilx["CAP_declared"] == 1) & (perfilx["CAP_true"] == 0)& (perfilx["CAP_max_pred"] == 1)])

110

In [126]:
df_udo["real"] = df_udo["ingreso_neto_comprobado"]

In [127]:
del df_udo["ingreso_neto_comprobado"]

In [128]:
for col in df_udo:
    if "ing" in col:
        df_udo[f"capacidad_{col}"] = (df_udo[col] - df_udo['descuentos']) / (df_udo["Monto_de_la_mensualidad"])
        df_udo[f"CAP_{col}"]= np.where((df_udo[f"capacidad_{col}"]>=2), 1,0)
        perfilx = df_udo[df_udo["BC_Score_"]>=680]
        print(f"CAP_{col}")
        print(len(perfilx[(perfilx["CAP_declared"] == 1) & (perfilx["CAP_true"] == 0)& (perfilx[f"CAP_{col}"] == 1)]))
        df_udo[f"CAP_{col}"]= np.where((df_udo[f"CAP_{col}"]==0), "Desaprobado","Aprobado")
        

CAP_ingreso_hipotecario
14
CAP_ingreso_prestamo
19
CAP_ingreso_tdc
33
CAP_ingreso_tarjetas_no_bancarias
8


In [129]:
df_udo["ingreso_neto_comprobado"] = df_udo["real"] 
del df_udo["real"]

In [130]:
#df_udo["rule"]= np.where((df_udo["rule"]==0), "otros","Perfil_X")
df_udo["CAP_true"]= np.where((df_udo["CAP_true"]==0), "Desaprobado","Aprobado")
df_udo["CAP_declared"]= np.where((df_udo["CAP_declared"]==0), "Desaprobado","Aprobado")
df_udo["CAP_pred"]= np.where((df_udo["CAP_pred"]==0), "Desaprobado","Aprobado")

In [131]:
df_udo["perfil"] = df_udo["rule"]

In [132]:
mask_desaprobado_perfil = (df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")

In [133]:
len(df_udo[mask_desaprobado_perfil & (df_udo["CAP_declared"] == "Aprobado")])

116

In [134]:
declarado_aprovado =  (df_udo["CAP_declared"] == "Aprobado") 
len(df_udo[mask_desaprobado_perfil & declarado_aprovado & (df_udo["CAP_pred"] == "Desaprobado")])

20

In [135]:
len(df_udo[mask_desaprobado_perfil & declarado_aprovado & (df_udo["CAP_max_pred"] == "Desaprobado")])

6

In [136]:
len(df_udo[(df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")& (df_udo["CAP_declared"] == "Aprobado")])

116

In [137]:
len(df_udo[(df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")& (df_udo["CAP_declared"] == "Aprobado")])

116

In [138]:
len(df_udo[(df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")& (df_udo["CAP_pred"] == "Desaprobado")])

286

In [139]:
len(df_udo[(df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")& (df_udo["CAP_max_pred"] == "Desaprobado")])

169

In [140]:
len(df_udo[(df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")& (df_udo["CAP_ingreso_hipotecario"] == "Desaprobado")])

336

In [141]:
columnas = [col for col in df_udo if "ing" in col] + [col for col in df_udo if "CAP" in col] + [col for col in df_udo if "ML" in col] + ["net_income_verified","ingreso_neto_comprobado", "perfil"]

In [142]:
#df_udo.columns

In [146]:
ingreso_validado = df_udo[["Consecutivo","BC_Score_",
'perfil',
'ingreso_neto_comprobado',
'net_income_verified',
'ML_declarado', 
'ML_nodeclarado',
'ML_income',
        'max_ML',
        "CAP_max_pred",        
'ingreso_hipotecario', 
'ingreso_prestamo',
'ingreso_tdc', 
'ingreso_tarjetas_no_bancarias',
'capacidad_ingreso_hipotecario', 
'CAP_ingreso_hipotecario',
'capacidad_ingreso_prestamo', 
'CAP_ingreso_prestamo',
'capacidad_ingreso_tdc', 
'CAP_ingreso_tdc',
'capacidad_ingreso_tarjetas_no_bancarias',
'CAP_ingreso_tarjetas_no_bancarias',
'CAP_true',
'CAP_declared',
'CAP_pred',  
'CAP_ingreso_hipotecario',
'CAP_ingreso_prestamo',
'CAP_ingreso_tdc',
'CAP_ingreso_tarjetas_no_bancarias']]

In [147]:
capacidad = [col for col in ingreso_validado if "CAP" in col and "declared" not in col]

In [148]:
ing_cap = ingreso_validado[capacidad]

In [149]:
cols = ing_cap.columns.to_numpy()
ingreso_validado["Aprobados"] = [cols[x].tolist() for x in ing_cap.eq('Aprobado').to_numpy()]

In [150]:
ingreso_validado["Aprobados"] = [','.join(map(str, l)) for l in ingreso_validado["Aprobados"]]

In [151]:
ingreso_validado["Aprobados"].isna().sum()

0

In [165]:
ingreso_validado.to_excel("validador_ingresos.xlsx")
       

In [153]:
perfil = ingreso_validado[ingreso_validado["perfil"] == "Perfil_X"]

In [162]:
criticos = perfil[(perfil["CAP_true"]=="Desaprobado") & (perfil["CAP_declared"]=="Aprobado")]

In [166]:
criticos.to_excel("criticos.xlsx")

In [157]:
perfil[(perfil["CAP_true"] == "Desaprobado") & perfil["CAP_declared"]=="Aprobado"]

,Consecutivo,BC_Score_,perfil,ingreso_neto_comprobado,net_income_verified,ML_declarado,ML_nodeclarado,ML_income,max_ML,CAP_max_pred,...,capacidad_ingreso_tarjetas_no_bancarias,CAP_ingreso_tarjetas_no_bancarias,CAP_true,CAP_declared,CAP_pred,CAP_ingreso_hipotecario,CAP_ingreso_prestamo,CAP_ingreso_tdc,CAP_ingreso_tarjetas_no_bancarias,Aprobados


In [154]:
perfil["Aprobados"]

0        CAP_max_pred,CAP_ingreso_tdc,CAP_ingreso_tdc,C...
2                           CAP_max_pred,CAP_true,CAP_pred
3        CAP_max_pred,CAP_ingreso_prestamo,CAP_ingreso_...
4        CAP_max_pred,CAP_ingreso_hipotecario,CAP_ingre...
7        CAP_max_pred,CAP_ingreso_tdc,CAP_ingreso_tdc,C...
                               ...                        
13928    CAP_max_pred,CAP_ingreso_prestamo,CAP_ingreso_...
13930    CAP_max_pred,CAP_ingreso_tdc,CAP_ingreso_tdc,C...
13933                       CAP_max_pred,CAP_true,CAP_pred
13934                       CAP_max_pred,CAP_true,CAP_pred
13938                       CAP_max_pred,CAP_true,CAP_pred
Name: Aprobados, Length: 7157, dtype: object

In [104]:
perfil["Aprobados"].astype(str).unique()

array(["['CAP_max_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_tdc', 'CAP_ingreso_tdc', 'CAP_true', 'CAP_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_tdc', 'CAP_ingreso_tdc']",
       "['CAP_max_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_comprobado', 'CAP_true', 'CAP_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_comprobado']",
       "['CAP_max_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_prestamo', 'CAP_ingreso_prestamo', 'CAP_ingreso_tdc', 'CAP_ingreso_tdc', 'CAP_ingreso_tarjetas_no_bancarias', 'CAP_ingreso_tarjetas_no_bancarias', 'CAP_true', 'CAP_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_prestamo', 'CAP_ingreso_prestamo', 'CAP_ingreso_tdc', 'CAP_ingreso_tdc', 'CAP_ingreso_tarjetas_no_bancarias', 'CAP_ingreso_tarjetas_no_bancarias']",
       "['CAP_max_pred', 'CAP_ingreso_neto_comprobado', 'CAP_ingreso_neto_compr

In [ ]:
for col in ingreso_validado.columns:
    

In [ ]:
df = ingreso_validado.apply(lambda x:x == "Aprobado")

In [ ]:
prueba.index = prueba.Consecutivo

In [ ]:
df_udo.eq('Aprobado').dot(df.columns + ',').str[:-1].str.split(',')

In [ ]:
prueba["col6"]

In [ ]:
for col in prueba.columns:
    if prueba[col] == True:
        prueba

In [ ]:
for x in list(ingreso_validado["Consecutivo"]):
    prueba["Consecutivo"]

In [ ]:
prueba = prueba.astype(int)

In [ ]:
prueba

In [ ]:
capacidad = [col for col in df_udo.columns if "CAP" in col]

In [ ]:
df_perfil_desprobado = (df_udo[(df_udo["CAP_true"] == "Desaprobado") & (df_udo["perfil"] == "Perfil_X")])

In [ ]:
cap_df = df_perfil_desprobado[capacidad+["Consecutivo"]]

In [ ]:
data = pd.DataFrame()
data["Consecutivo"] = cap_df["Consecutivo"]

In [ ]:
data.index = data["Consecutivo"] 

In [ ]:
[col for col in cap_df.columns if cap_df[col]]

In [ ]:
np.where((cap_df=="Aprobado"), 1,0)

In [ ]:

for col in cap_df.columns:
    if cap_df[col] == "Aprobado":
        data["Aprobado"] = col

In [ ]:
c = cap_df.columns[(cap_df == "Aprobado").all()]
print (c)
#Index(['Consecutivo'], dtype='object')

## * 2. agregarle excel de adrian
## * 3. Agrupar por bureau (quantiles) y sacar todo promedio (CAP; ML;real;pred)
## * 4. Salvar predicción con el parametrico? Validar el ingreso de alguna manera 

# Analisis de Capacidad de pago



In [ ]:
df2.groupby(["CAP_true"], as_index=False).agg(
    cantidad =  ('CAP_true', 'count'))

In [ ]:
df2.groupby(["CAP_declared"], as_index=False).agg(
    cantidad =  ('CAP_true', 'count'))

In [ ]:
matrix = df2.groupby(["CAP_true","CAP_declared"], as_index=False).agg(
    cantidad =  ('CAP_true', 'count'))
matrix = pd.pivot_table(matrix,index=["CAP_true"],values="cantidad",columns="CAP_declared")
matrix

Observamos que del total del dataset

In [ ]:
len(df2)

## Matrices de confusion

Matriz de confusión segun capacidad de pago declarada

In [ ]:
cm = pd.crosstab(df2["CAP_true"], df2["CAP_declared"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Blues", fmt='g')

# Perfil x

## Analisis con ingreso declarado y predicho

### Comparando para tener la misma cantidad de conversion

In [ ]:
perfil_x = df2[df2["rule"]=="Perfil_X"]

In [ ]:
len(perfil_x)/len(df2)

In [ ]:
cantidad = len(perfil_x[perfil_x["CAP_declared"] == "Aprobado"])
cantidad

In [ ]:
len(df2)

In [ ]:
#df2[["Consecutivo","net_income_verified","ingreso_neto_comprobado","ML_income","BC_Score_"]].to_csv("ingresos.csv")

In [ ]:
perfilx = df2[df2["BC_Score_"]>=680]
cm = pd.crosstab(perfilx["CAP_true"], perfilx["CAP_declared"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Oranges", fmt='g')

In [ ]:

cm = pd.crosstab(perfilx["CAP_true"], perfilx["CAP_pred"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Oranges", fmt='g')

In [ ]:
aprobados_por_ambos = perfilx[(perfilx["CAP_pred"] == "Aprobado") & (perfilx["CAP_declared"] == "Aprobado")]

In [ ]:
len(aprobados_por_ambos)

In [ ]:
11806+1653+126+355

In [ ]:
1653+355

In [ ]:
126/13940 *100

In [ ]:
796+286

In [ ]:
1082/7157 *100

In [ ]:
85

In [ ]:
2008/13940

In [ ]:
len(perfilx[perfilx["CAP_declared"] == "Aprobado"])

In [ ]:
perfilx_df.ML_income.mean()

In [ ]:
perfilx_df.net_income_verified.mean()

In [ ]:
perfilx_df.ingreso_neto_comprobado.mean()

In [ ]:
perfilx_df.ingreso_neto_comprobado.max()

In [ ]:
cm = pd.crosstab(perfilx_df["CAP_true"], perfilx_df["CAP_pred"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Blues", fmt='g')

In [ ]:
#pd.read_excel("GRAFICAS_2022-03-29.xlsx")

In [ ]:
df2["bins_bc"] = pd.cut(df2["BC_Score_"], bins=range(0,1000,100))

In [ ]:
df2.groupby("bins_bc").agg(
    mean_ML_income =  ('ML_income', 'mean'),
    mean_true_income = ('ingreso_neto_comprobado', 'mean'),  mean_declarado = ('net_income_verified', 'mean'))


In [ ]:
errores_modelo = perfilx_df[(perfilx_df["CAP_true"]=="Desaprobado") & (perfilx_df["CAP_pred"]=="Aprobado")]

In [ ]:
df_noventa = pd.merge(adrian,errores_modelo,how="inner", left_on="Consecutivo",right_on="Consecutivo")

In [ ]:
df_noventa = df_noventa[['ingreso_hipotecario',
 'ingreso_prestamo',
 'ingreso_tdc',
 'ingreso_tarjetas_no_bancarias',"Consecutivo","ML_income","CAP_true","CAP_pred","CAP_declared","net_income_verified","ingreso_neto_comprobado","BC_Score_"]]

In [ ]:
df2["CAP_perfilX"]= np.where((df2["capacidad_pago_declarada"]>=2) &(df2["BC_Score_"]>=680 ), "Aprobado","Desaprobado")

In [ ]:
#perf = df2[df2["valor_del_score"]>=680]

In [ ]:
cm = pd.crosstab(df2["CAP_true"], df2["CAP_perfilX"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Oranges", fmt='g')

In [ ]:
print(cantidad)

In [ ]:
errores = len(df2[(df2["CAP_true"]=="Desaprobado") & (df2["CAP_perfilX"]=="Aprobado")])
errores

In [ ]:
errores/len(df2) * 100

### Misma cantidad de conversion

In [ ]:
for umbral in np.linspace(3,4,100):
    df2[f"CAP_pred{int(umbral)}"]= np.where((df2["capacidad_pago_predicha"]>=umbral), "Aprobado","Desaprobado")
    cant = len(df2[df2[f"CAP_pred{int(umbral)}"]=="Aprobado"])
    if cant+1<=cantidad:
        print(umbral,cant)
        print(int(umbral))
        break
        
cm = pd.crosstab(df2["CAP_true"], df2[f"CAP_pred{int(umbral)}"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="tab:blue", cmap="Greens", fmt='g')

In [ ]:
pred_aprobados = len(df2[(df2[f"CAP_pred{int(umbral)}"]=="Aprobado")])
pred_aprobados/len(df2) *100

In [ ]:
errores_model = len(df2[(df2[f"CAP_pred{int(umbral)}"]=="Aprobado") & (df2["CAP_true"]=="Desaprobado")])


In [ ]:
errores_model/len(df2) *100

In [ ]:
reduccion = (errores - errores_model)/errores 
reduccion

In [ ]:
(1- reduccion)*errores

### aumentar la conversión y mantener error

In [ ]:
len(df2[(df2[f"CAP_pred"]=="Aprobado") & (df2[f"CAP_true"]=="Desaprobado")])

In [ ]:
for umbral in np.linspace(1.8,2.1,100):
    df2[f"CAP_pred{int(umbral)}"]= np.where((df2["capacidad_pago_predicha"]>=umbral), "Aprobado","Desaprobado")
    cant = len(df2[(df2[f"CAP_pred{int(umbral)}"]=="Aprobado") & (df2[f"CAP_true"]=="Desaprobado")])
    if cant<=errores:
        print(umbral,cant)
        print(int(umbral))
        break
        
cm = pd.crosstab(df2["CAP_true"], df2[f"CAP_pred{int(umbral)}"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="tab:blue", cmap="Greens", fmt='g')


In [ ]:
len(df2[(df2[f"CAP_pred{int(umbral)}"]=="Aprobado")])

In [ ]:
cant_perfilx = len(perfil_x[df2["CAP_perfilX"] == "Aprobado"])
cant_perfilx

In [ ]:
cant_cap = len(df2[(df2[f"CAP_pred{int(umbral)}"]=="Aprobado")])
cant_cap

In [ ]:
(cant_cap - cant_perfilx)/cant_perfilx

In [ ]:
cant_perfilx * 1.7

In [ ]:
errores/cant_cap * 100


### Punto de corte 2

In [ ]:
cm = pd.crosstab(df2["CAP_true"], df2["CAP_pred"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Blues", fmt='g')

In [ ]:
(df2["CAP_pred"] == "Aprobado").sum()

In [ ]:
(df2["CAP_pred"] == "Aprobado").sum()/len(df2)

In [ ]:
len(df2[(df2["CAP_pred"] == "Aprobado")& (df2["CAP_true"] == "Desaprobado")])/(df2["CAP_pred"] == "Aprobado").sum() *100

### Score mayor a 680 con modelo de ingreso

In [ ]:
df2["CAP_modelX"]= np.where((df2["capacidad_pago_predicha"]>=2) &(df2["BC_Score_"]>=680 ), "Aprobado","Desaprobado")

In [ ]:
#perf = df2[df2["valor_del_score"]>=680]

In [ ]:
cm = pd.crosstab(df2["CAP_true"], df2["CAP_modelX"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Oranges", fmt='g')

In [ ]:
len(df2)

In [ ]:
(df2["CAP_modelX"] == "Aprobado").sum()

In [ ]:
(df2["CAP_modelX"] == "Aprobado").sum()/len(df2)

In [ ]:
+ (116-96)/116

In [ ]:
(len((df2[(df2["CAP_modelX"] == "Aprobado")&(df2["CAP_true"] == "Desaprobado") ]))/(df2["CAP_modelX"] == "Aprobado").sum() )*100

## Perfil X con CAP 1.8

In [ ]:
df2["CAP_perfilX_1.8"]= np.where((df2["capacidad_pago_declarada"]>=1.8) &(df2["BC_Score_"]>=680 ), "Aprobado","Desaprobado")

In [ ]:
df2["CAP_true1.8"]= np.where((df2["capacidad_pago_real"]>=1.8), 1,0)
df2["CAP_true1.8"]= np.where((df2["CAP_true1.8"]==0), "Desaprobado","Aprobado")

In [ ]:
cm = pd.crosstab(df2["CAP_true1.8"], df2["CAP_perfilX_1.8"])
plt.rcParams['figure.facecolor'] = 'white'
fig, ax = plt.subplots(ncols=1, figsize=(5,5))
sns.heatmap(cm, 
                    annot=True,ax=ax,
                    linewidths=.2,linecolor="Darkblue", cmap="Oranges", fmt='g')

# Diferencias

$$
  Declaran = \frac{Declared Income - True Income}{True Income}
$$

In [ ]:
df2["diff_per"] = (df2["net_income_verified"] - df2["ingreso_neto_comprobado"])/df2["ingreso_neto_comprobado"]

In [ ]:
#dataframe.loc[(condicion),columnas en particular]

In [ ]:
df2.loc[df2["diff_per"] == 0,  'Declaran'] = "Lo Mismo"
df2.loc[(df2["diff_per"] < 0), 'Declaran'] = "Menos"
df2.loc[(df2["diff_per"] > 0) & (df2["diff_per"] <= 0.3), 'Declaran'] = "entre 1% y 30% más"
df2.loc[(df2["diff_per"] > 0.3) & (df2["diff_per"] <= 0.5), 'Declaran'] = "entre 31% y 50% más"
df2.loc[(df2["diff_per"] > 0.5) & (df2["diff_per"] < 1), 'Declaran'] = "entre 51% y 99% más"
df2.loc[(df2["diff_per"] >= 1) , 'Declaran'] = "Más del 100%"

In [ ]:
reorderlist = ["Más del 100%","entre 51% y 99% más","entre 31% y 50% más","entre 1% y 30% más","Lo Mismo", "Menos"]


In [ ]:
declararon = pd.merge(df2.Declaran.value_counts(),df2.Declaran.value_counts(normalize=True), right_index=True, left_index=True)
declararon.columns = ["Cantidad Clientes","Proporcion"]
declararon = declararon.reindex(reorderlist)
declararon.index.name = "Declaran"


In [ ]:
plt.figure(figsize=(10,10))
paleta = ["#d53e4f","#fc8d59", "#fee08b", "#e6f598", "#99d594", "#3288bd"]
sns.barplot(x="Cantidad Clientes",y=declararon.index,data=declararon, palette=paleta)
plt.savefig("cantidad.png", bbox_inches='tight')

In [ ]:
declararon.loc['Total',:]= declararon.sum(axis=0)

per_problema = declararon[0:3]["Proporcion"].sum(axis=0)

declararon["Proporcion"] = (declararon["Proporcion"].round(2)*100).astype(str) + '%'
declararon["Cantidad Clientes"] = declararon["Cantidad Clientes"].astype(int)

In [ ]:
total_problema = declararon[0:3]["Cantidad Clientes"].sum(axis=0)

print(f"El gran problema representa {total_problema} clientes \ny representa el {(per_problema*100):.2f}% de la población que \nconocemos el ingreso real \n")
declararon

## Predicciones 

In [ ]:
df2 = core.copy()

### %_validated

La respuesta de nuestra API también considera % de validado cómo la división entre el ingreso predicho cómo númerador y el ingreso real cómo denominador. <br>

Entendemos que nunca el ingreso predicho debe ser mayor al ingreso declarado. A lo sumo debería ser 100%.

Realizamos entonces la variable % validado y luego analizamos en que decil cae entre 0 y 1. Creamos diez categorías y vemos que % de la muestra representa


In [ ]:
df2["%_validated"] = df2["ML_income"]/df2["net_income_verified"]


lst = [df2]
for column in lst:
    column.loc[(column["%_validated"] >= 0 ) & (column["%_validated"] <= 0.1), 'dec_%_validado'] = "0 hasta 10%"
    column.loc[(column["%_validated"] > 0.1) & (column["%_validated"] <= 0.2), 'dec_%_validado'] = "11 hasta 20%"
    column.loc[(column["%_validated"] > 0.2) & (column["%_validated"] <= 0.3), 'dec_%_validado'] = "21 hasta 30%"
    column.loc[(column["%_validated"] > 0.3) & (column["%_validated"] <= 0.4), 'dec_%_validado'] = "31 hasta 40%"
    column.loc[(column["%_validated"] > 0.4) & (column["%_validated"] <= 0.5), 'dec_%_validado'] = "41 hasta 50%"
    column.loc[(column["%_validated"] > 0.5) & (column["%_validated"] <= 0.6), 'dec_%_validado'] = "51 hasta 60%"
    column.loc[(column["%_validated"] > 0.6) & (column["%_validated"] <= 0.7), 'dec_%_validado'] = "61 hasta 70%"
    column.loc[(column["%_validated"] > 0.7) & (column["%_validated"] <= 0.8), 'dec_%_validado'] = "71 hasta 80%"
    column.loc[(column["%_validated"] > 0.8) & (column["%_validated"] <= 0.9), 'dec_%_validado'] = "81 hasta 90%"
    column.loc[(column["%_validated"] > 0.9) & (column["%_validated"] < 0.1), 'dec_%_validado'] = "91 hasta 99%"
    column.loc[column["%_validated"] == 1, 'salarios_min'] = "+100% validado"

In [ ]:
ax = df2["dec_%_validado"].value_counts(normalize=True).sort_index().plot.barh()
ax.set_ylabel('% de ingreso declarado valido  \n ')
ax.set_title('Distribución de la predicción \n ')

plt.show()

### Ingresos Mínimos

Según la fuente https://www.gob.mx/stps/prensa/comunicado-conjunto-289406. El ingreso mínimo durante el 2021 fue de 4.000 pesos mexicanos. Por lo tanto en vez de separar la muestra en quantiles, decidimos discretizar según está variable más interpretable

In [ ]:
df2["ratio_income_min_w"] = df2["ingreso_neto_comprobado"]/4000

lst = [df2]
for column in lst:
    column.loc[column["ratio_income_min_w"] <= 1,  'salarios_min'] = " Hasta 1 salario min"
    column.loc[(column["ratio_income_min_w"] > 1) & (column["ratio_income_min_w"] <= 2), 'salarios_min'] = "1 hasta 2 salarios"
    column.loc[(column["ratio_income_min_w"] > 2) & (column["ratio_income_min_w"] <= 3), 'salarios_min'] = "2 hasta 3 salarios"
    column.loc[(column["ratio_income_min_w"] > 3) & (column["ratio_income_min_w"] <= 5), 'salarios_min'] = "3 hasta 5 salarios"
    column.loc[column["ratio_income_min_w"] >5, 'salarios_min'] = "+ de 5 salarios"
    
reorderlist = [" Hasta 1 salario min", "1 hasta 2 salarios","2 hasta 3 salarios","3 hasta 5 salarios", "+ de 5 salarios"]


In [ ]:
cantidad = df2.groupby(["salarios_min"]).agg(
    Cantidad =  ('salarios_min', 'count')).reindex(reorderlist)
proporcion = df2.salarios_min.value_counts(normalize=True).reindex(reorderlist).round(3)
inegi_df = pd.merge(cantidad,proporcion, right_index=True, left_index=True)
inegi_df.columns = ["Cantidad Clientes Kavak","Proporcion Muestra"]
inegi_df = inegi_df.reindex(reorderlist)
inegi_df.loc['Total',:]= inegi_df.sum(axis=0)
inegi_df.fillna(0, inplace=True)


inegi_df.index.name = "Ingreso Real"
inegi_df["Mexico"] = [10642543,17141160,9818858,4522674,1741491,43866726]
inegi_df["% Poblacion Mexico"] = [valor/43866726 for valor in inegi_df["Mexico"]]
inegi_plot = inegi_df.copy()
inegi_df["% Poblacion Mexico"] = ((inegi_df["% Poblacion Mexico"]*100).round(3)).astype(str) + '%'
inegi_df["Proporcion Muestra"] = ((inegi_df["Proporcion Muestra"]*100).round(2)).astype(str) + '%'
inegi_plot =inegi_plot.reset_index(level=0)
inegi_plot[0:5].plot(x="Ingreso Real",y=["Proporcion Muestra","% Poblacion Mexico"], kind="bar", width=0.8, alpha=0.8, align='center',figsize=(8,6)).set_ylim([0,0.6])
current_values = plt.gca().get_yticks()
# using format string '{:.0f}' here but you can choose others
plt.gca().set_yticklabels(['{:,.0%}'.format(x) for x in current_values])
plt.xticks(rotation=45)

plt.xlabel('Distribución Income')

inegi_df

Analizamos nuestra muestra versus la población de méxico. Observamos que en Kavak el 65% de la población se encuentra por encima de los 5 salarios observando una piramide invertida respecto a la población méxicana extraída del INEGI 2019 

## % validado por ingreso mínimo

Realizamos la variable cantidad de ingresos mínimos para encontrar una interepretación

In [ ]:
df2["ratio_income_min_w"] = df2["ingreso_neto_comprobado"]/4000

lst = [df2]
for column in lst:
    column.loc[column["ratio_income_min_w"] <= 1,  'salarios_min'] = " Hasta 1 salario min"
    column.loc[(column["ratio_income_min_w"] > 1) & (column["ratio_income_min_w"] <= 2), 'salarios_min'] = "1 hasta 2 salarios"
    column.loc[(column["ratio_income_min_w"] > 2) & (column["ratio_income_min_w"] <= 3), 'salarios_min'] = "2 hasta 3 salarios"
    column.loc[(column["ratio_income_min_w"] > 3) & (column["ratio_income_min_w"] <= 5), 'salarios_min'] = "3 hasta 5 salarios"
    column.loc[(column["ratio_income_min_w"] > 5) & (column["ratio_income_min_w"] <= 8), 'salarios_min'] = "5 hasta 8 salarios"
    column.loc[(column["ratio_income_min_w"] > 8) & (column["ratio_income_min_w"] <= 12), 'salarios_min'] = "8 hasta 12 salarios"
    column.loc[column["ratio_income_min_w"] >12, 'pred_min'] = "+ de 12 salarios"

    
    
reorderlist = [ "1 hasta 2 salarios","2 hasta 3 salarios","3 hasta 5 salarios", "5 hasta 8 salarios","8 hasta 12 salarios","+ de 12 salarios"]

reorderlist2 = ["8 hasta 12 salarios","5 hasta 8 salarios", "3 hasta 5 salarios","2 hasta 3 salarios", "1 hasta 2 salarios"]


Este gráfico analiza el % de ingreso declarado válidado por la predicción del modelo. Vemos que el caso de ingresos de 1 hasta 2 salarios mínimos es donde encontramos mayores desvios en donde en un 16% de la muestar podemos validar sólo hasta el 30% de lo declarado y un 33% hasta un 70% de lo declarado. Vemos que también algo sucede con los mayores salarios.

Analizamos que el 40% del muestra se encuentra con un ingreso entre 81% hasta 90% validado. Ningún cliente se encuentra con el 100% de los ingresos validados. Esa es una elección de nuestro modelo para subestimar el ingreso

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
reorderlist2 = ["8 hasta 12 salarios","5 hasta 8 salarios", "3 hasta 5 salarios","2 hasta 3 salarios", "1 hasta 2 salarios"]

cross = pd.crosstab(df2["salarios_min"] ,df2["dec_%_validado"]).apply(lambda x: round(x/x.sum(),3), axis=1).reindex(reorderlist2)
#sns.heatmap(cross, annot=True)

cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True).reversed()
# plot heatmap
sns.despine(left=True)
sns.heatmap(cross, fmt=".2%",annot=True,annot_kws={"size": 12},
           linewidths=5, cmap=cmap,cbar=False, square=True)


plt.rcParams["axes.labelsize"] = 10
plt.xlabel('% Validado predicho \n ', fontsize = 10) # x-axis label with fontsize 15
plt.ylabel('Ingreso real \n', fontsize = 10)

plt.title("\n % Validado predicho vs Verdadero \n100% predicho por bucket en comparación a real\n", fontsize = 12)


## Evaluación del modelo

In [ ]:
df2["ration_pred_min"] = df2["ML_income"]/4000


lst = [df2]
for column in lst:
    column.loc[column["ration_pred_min"] <= 1,  'pred_min'] = " Hasta 1 salario min"
    column.loc[(column["ration_pred_min"] > 1) & (column["ratio_income_min_w"] <= 2), 'pred_min'] = "1 hasta 2 salarios"
    column.loc[(column["ration_pred_min"] > 2) & (column["ratio_income_min_w"] <= 3), 'pred_min'] = "2 hasta 3 salarios"
    column.loc[(column["ration_pred_min"] > 3) & (column["ratio_income_min_w"] <= 5), 'pred_min'] = "3 hasta 5 salarios"
    column.loc[(column["ration_pred_min"] > 5) & (column["ratio_income_min_w"] <= 8), 'pred_min'] = "5 hasta 8 salarios"
    column.loc[(column["ration_pred_min"] > 8) & (column["ratio_income_min_w"] <= 12), 'pred_min'] = "8 hasta 12 salarios"
    column.loc[column["ration_pred_min"] >12, 'pred_min'] = "+ de 12 salarios"
    
reorderlist = [ "1 hasta 2 salarios","2 hasta 3 salarios","3 hasta 5 salarios", "5 hasta 8 salarios","8 hasta 12 salarios","+ de 12 salarios"]

ingreso_diff_per = pd.crosstab(df2['salarios_min'],df2['pred_min']).apply(lambda x: x/x.sum(), axis=1).reindex(reorderlist2)
ingreso_diff_per = ingreso_diff_per[reorderlist]

    


fig, ax = plt.subplots(figsize=(10, 10))
label_size = 8
plt.rcParams['xtick.labelsize'] = label_size
plt.rcParams['ytick.labelsize'] = label_size
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True).reversed()
# plot heatmap
sns.despine(left=True)
sns.heatmap(ingreso_diff_per, fmt=".2%",annot=True,annot_kws={"size": 12},
           linewidths=5, cmap=cmap,cbar=False, square=True)

plt.rcParams["axes.labelsize"] = 10
plt.xlabel('Salario Real \n ', fontsize = 10) # x-axis label with fontsize 15
plt.ylabel('Predicho \n', fontsize = 10)

plt.suptitle("Evaluación del Modelo \n\n ", fontsize = 14)
plt.title("\n \n\n Ingreso predicho vs Verdadero \n100% predicho por bucket en comparación a real\n", fontsize = 12)
plt.savefig("figures/evaluacion_salario_porcentaje.png", bbox_inches='tight' )

Del 100% de la predicción discretizada por ingresos mínimo. ¿Cuantos nos equivocamos? ¿A qué distancia del bucket caen?

El resultado perfecto sería la diagonal en 100%. Lo que menos esperamos es sobre estimar y en caso de hacerlo no pasarnos en más de un bucket.

## Interpretabilidad del modelo

In [ ]:
import pickle
from s3_utils.s3_utils import write_parquet_from_pd, read_pd_from_parquet, start_logger,write_pickle,read_pickle
path_s3 = "s3://data-science-kavak-dev/projects/cerberus/v2/dev/income/models/"

predictor_decla = read_pickle(path_s3 +"declarado.pkl")
boost = predictor_decla.booster_
columns_declarado=boost.feature_name()
X_declarado = df2[columns_declarado]

In [ ]:
predictor_sin = read_pickle(path_s3 +"nodeclarado.pkl")
boost = predictor_sin.booster_
columns_sindeclarado=boost.feature_name()
X_sindeclarado = df2[columns_sindeclarado]

In [ ]:
#pip install --upgrade pyspark


In [ ]:
#explainer2 = shap.Explainer(predictor_sin)
import shap


In [ ]:
explainer = shap.TreeExplainer(predictor_sin)

In [ ]:

explainer_decla = shap.TreeExplainer(predictor_decla)

In [ ]:

df = df2.copy()
df["ML_Sin_declared"] = df2["ML_nodeclarado"]

In [ ]:
df["ML_declared"] = df["ML_declarado"]

In [ ]:
df["min_models"] = df["ML_income"]
df["ML_income"] = df["ML_income"].astype(int)

In [ ]:
df["%_validado"] = df["ML_income"]/df["net_income_verified"]

In [ ]:
df["%_validado"] = df[["%_validado"]].applymap(lambda x: "{0:.1f}%".format(x*100))

In [ ]:
df["true_income"] =df["ingreso_neto_comprobado"]

In [ ]:
df['declared']=df["net_income_verified"]

In [ ]:
df["validado"] = df["min_models"]/df["net_income_verified"]
df[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'min_models','valor_del_score']] = df[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'min_models','valor_del_score']].astype(int)

In [ ]:
df_shap = df.loc[(df["validado"] < 0.8) &(df["declared"]> 1.4*df["true_income"]) & (df["ML_income"]<=df["declared"]),]

In [ ]:
df_shap[3:4][['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'ML_income','BC_Score_',"%_validado"]].T

# Ejemplo 1

In [ ]:
df_shap[3:4][['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'ML_income','BC_Score_',"%_validado"]].T

In [ ]:
df_shap[5:6][['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'ML_income','BC_Score_',"%_validado"]].T

In [ ]:
df_shap= df_shap[0:10]
shap_values2 = explainer(df_shap[columns_sindeclarado])
shap_values_decla = explainer_decla(df_shap[columns_declarado])

In [ ]:
shap.plots.waterfall(shap_values_decla[6])

In [ ]:
shap.plots.waterfall(shap_values2[6])

### Ejemplo 2

In [ ]:

df_shap = df.loc[(df["declared"]>df["true_income"]) & (df["min_models"]<=df["true_income"]),]
df_shap =  df_shap[df_shap["validado"] < 0.4]
df_shap = df_shap[0:25]


In [ ]:
shap_values2 = explainer(df_shap[columns_sindeclarado])
shap_values_decla = explainer_decla(df_shap[columns_declarado])

In [ ]:
df_shap[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'ML_income','BC_Score_',"%_validado","validado"]][5:6].T

In [ ]:
shap.plots.waterfall(shap_values_decla[5])

In [ ]:
shap.plots.waterfall(shap_values2[5])

### Ejemplo 3

In [ ]:
#df_shap = df.loc[(df["declared"]== df["true_income"])]
df_shap =  df[df["validado"] == 1 ]
#df_shap = df_shap[0:25]


In [ ]:
df_shap = df_shap.loc[(df_shap["min_models"]>df_shap["true_income"]) & (df_shap['valor_del_score']>680),]

In [ ]:
df_shap[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'ML_income','BC_Score_',"%_validado","validado"]][0:1].T

In [ ]:
shap_values2 = explainer(df_shap[columns_sindeclarado])
shap_values_decla = explainer_decla(df_shap[columns_declarado])

In [ ]:
shap.plots.waterfall(shap_values_decla[0])

In [ ]:
shap.plots.waterfall(shap_values2[0])

# Ejemplo 4

In [ ]:
df_shap =  df.loc[(df["declared"]<df["ML_declared"]) & (df["declared"]<df["ML_Sin_declared"]), ]
df_shap[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'min_models','BC_Score_',"%_validado","validado"]][1:2].T

In [ ]:
df_shap = df_shap[0:5]

In [ ]:
shap_values2 = explainer(df_shap[columns_sindeclarado])
shap_values_decla = explainer_decla(df_shap[columns_declarado])

In [ ]:
shap.plots.waterfall(shap_values_decla[1])

In [ ]:
shap.plots.waterfall(shap_values2[1])

# Ejemplo 5

In [ ]:
#df_shap = df.loc[(df["declared"]== df["true_income"])]
df_shap =  df[df["validado"] == 1 ]
df_shap = df_shap.loc[(df_shap["declared"]>30000),]
df_shap = df_shap.loc[(df_shap["min_models"]>df_shap["true_income"]) & (df_shap['BC_Score_']>680),]
#df_shap = df_shap.loc[(df_shap["true_income"]<14000)  ,]
df_shap[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'min_models','BC_Score_',"%_validado","validado"]][2:3].T

In [ ]:
df_shap = df_shap[2:3]
shap_values2 = explainer(df_shap[columns_sindeclarado])
shap_values_decla = explainer_decla(df_shap[columns_declarado])

In [ ]:
shap.plots.waterfall(shap_values_decla[0])

In [ ]:
shap.plots.waterfall(shap_values2[0])

## Ejemplo 6

In [ ]:
#df_shap = df.loc[(df["declared"]== df["true_income"])]
df_shap =  df[df["validado"] == 1 ]
df_shap = df_shap.loc[(df_shap["min_models"]<28000),]
df_shap = df_shap.loc[(df_shap["min_models"]>df_shap["true_income"]) & (df_shap['valor_del_score']>680),]
df_shap = df_shap.loc[(df_shap["true_income"]<14000)  ,]
df_shap[['Consecutivo','declared','true_income','ML_declared','ML_Sin_declared', 'min_models','BC_Score_',"%_validado","validado"]][4:5].T

In [ ]:
df_shap = df_shap[4:5]

In [ ]:
shap_values2 = explainer(df_shap[columns_sindeclarado])
shap_values_decla = explainer_decla(df_shap[columns_declarado])

In [ ]:
shap.plots.waterfall(shap_values_decla[0])

In [ ]:
shap.plots.waterfall(shap_values2[0])

## Performance


# Performance

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error as MAPE_skl
MAPE_skl

In [ ]:
MAPE_skl(df["true_income"],df["ML_declared"]).round(3) 

In [ ]:
MAPE_skl(df["true_income"],df["ML_nodeclarado"]).round(3) 

In [ ]:
MAPE_skl(df["true_income"],df["net_income_verified"]).round(3) 

In [ ]:
MAPE_skl(df["true_income"],df["ML_income"]).round(3) 

In [ ]:
df_map = df.query("true_income<100000")

In [ ]:
MAPE_skl(df_map["true_income"],df_map["ML_income"]).round(3) 

# Anexo:

Variables de bureau de crédito utilizadas para el modelo

In [ ]:
variables_bureau = ['fecha_de_apertura_de_la_cuenta_más_antigua',
 'fecha_de_apertura_de_la_cuenta_más_reciente',
 'fecha_de_la_consulta_mas_reciente',
 'mensaje_de_alerta',
 'moneda_del_credito',
 'nueva_direccion_en_los_últimos_60_días',
 'número_de_cuentas',
 'número_de_cuentas_cerradas',
 'número_de_cuentas_con_historial_de_morosidad',
 'número_de_cuentas_con_mop_=_00',
 'número_de_cuentas_con_mop_=_01',
 'número_de_cuentas_con_mop_=_02',
 'número_de_cuentas_con_mop_=_03',
 'número_de_cuentas_con_mop_=_04',
 'número_de_cuentas_con_mop_=_05',
 'número_de_cuentas_con_mop_=_06',
 'número_de_cuentas_con_mop_=_07',
 'número_de_cuentas_con_mop_=_96',
 'número_de_cuentas_con_mop_=_97',
 'número_de_cuentas_con_mop_=_99',
 'número_de_cuentas_con_mop_=_UR',
 'número_de_cuentas_con_morosidad_actual',
 'número_de_cuentas_de_pagos_fijos_e_hipotecarios',
 'número_de_cuentas_en_aclaración',
 'número_de_cuentas_en_despacho_de_cobranza_o_administadora_de_cartera',
 'número_de_cuentas_revolventes_y_sin_límite_pre-establecido',
 'número_de_solicitudes_de_consulta',
 'número_de_solicitudes_del_informe_de_buró',
 'número_de_solicitudes_del_informe_de_buró_realizadas_por_despachos_de_cobranza_o_administadora_de_cartera',
 'porcentaje_del_límite_de_crédito_utilizado_para_cuentas_revolventes_y_sin_límite_pre-establecido',
 'total_de_creditos_máximos_para_cuentas_revolventes_y_sin_límite_pre-establecido',
 'total_de_créditos_máximos_para_cuentas_de_pagos_fijos_e_hipotecarios',
 'total_de_importe_de_pago_para_cuentas_de_pagos_fijos_e_hipotecarios',
 'total_de_importe_pagado_para_cuentas_revolventes_y_sin_límite_pre-establecido',
 'total_de_límites_de_crédito_para_cuentas_revolventes_y_sin_límite_pre-establecido',
 'total_de_saldos_actuales_para_cuentas_de_pagos_fijos_e_hipotecarios',
 'total_de_saldos_actuales_para_cuentas_revolventes_y_sin_límite_pre-establecido',
 'total_de_saldos_vencidos_para_cuentas_de_pagos_fijos_e_hipotecarios',
 'total_de_saldos_vencidos_para_cuentas_revolventes_y_sin_límite_pre-establecido',
 'código_de_razón_1',
 'código_de_razón_2',
 'código_de_razón_3',
 'código_del_score',
 'nombre_del_score',
 'valor_del_score']



In [ ]:
print(f"La cantidad de variables utilizadas para entrenamiento del modelo son {len(variables_bureau)}")

In [ ]:
variables_bureau

In [ ]:
!jupyter nbconvert analisis_resultados.ipynb --to html --TemplateExporter.exclude_input=True